In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

tf.random.set_seed(777)

In [2]:
#keras에서 제공하는 데이터셋
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 [==============================] - 9s 1us/step


In [4]:
word_index = imdb.get_word_index()
 
word_index = {k:(v+3) for k,v in word_index.items()}
word_index[""] = 0 #공백
word_index[""] = 1 #시작
word_index[""] = 2 #모르는 단어
word_index[""] = 3 #사용되지 않은 단어

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

decode_review(train_data[4])

"? worst mistake of my life br br i picked this movie up at target for 5 because i figured hey it's sandler i can get some cheap laughs i was wrong completely wrong mid way through the film all three of my friends were asleep and i was still suffering worst plot worst script worst movie i have ever seen i wanted to hit my head up against a wall for an hour then i'd stop and you know why because it felt damn good upon bashing my head in i stuck that damn movie in the ? and watched it burn and that felt better than anything else i've ever done it took american psycho army of darkness and kill bill just to get over that crap i hate you sandler for actually going through with this and ruining a whole day of my life"

In [5]:
print(train_labels[4])

0


In [6]:
#학습과 평가를 위해 256길이로 동일길이 선언
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index[""],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index[""],
                                                       padding='post',
                                                       maxlen=256)
print(len(train_data[0]), len(test_data[0]))
print(train_data[0])

256 256
[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  10

In [7]:
vocab_size = 10000

#모델 선언
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu)) #16개의 layer 지정
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

#모델 컴파일
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

2022-11-03 18:47:09.945808: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
#모델을 평가할 test 데이터 정의 - 10000을 기준으로 학습과 평가 수행
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]


history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/40
30/30 [==============================] - 1s 15ms/step - loss: 0.6923 - accuracy: 0.5381 - val_loss: 0.6909 - val_accuracy: 0.6784
Epoch 2/40
30/30 [==============================] - 0s 12ms/step - loss: 0.6880 - accuracy: 0.7215 - val_loss: 0.6848 - val_accuracy: 0.7143
Epoch 3/40
30/30 [==============================] - 0s 12ms/step - loss: 0.6785 - accuracy: 0.7497 - val_loss: 0.6719 - val_accuracy: 0.7370
Epoch 4/40
30/30 [==============================] - 0s 12ms/step - loss: 0.6598 - accuracy: 0.7657 - val_loss: 0.6501 - val_accuracy: 0.7647
Epoch 5/40
30/30 [==============================] - 0s 12ms/step - loss: 0.6310 - accuracy: 0.7833 - val_loss: 0.6187 - val_accuracy: 0.7680
Epoch 6/40
30/30 [==============================] - 0s 12ms/step - loss: 0.5926 - accuracy: 0.8020 - val_loss: 0.5807 - val_accuracy: 0.7970
Epoch 7/40
30/30 [==============================] - 0s 13ms/step - loss: 0.5478 - accuracy: 0.8205 - val_loss: 0.5384 - val_accuracy: 0.8128
Epoch 8/40
30

In [9]:
#테스트 데이터로 모델 검증
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 1s 767us/step - loss: 0.3273 - accuracy: 0.8731
[0.3272671401500702, 0.8730800151824951]
